In [10]:
import cv2
import numpy as np
import math as m
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
def nothing(x):
    print(x)
    
#Close the pop up images before running next cell. Skip to last cell for output

In [12]:
src_img = cv2.imread('screw11.jpg',1)
cv2.imshow('read',src_img)
cv2.waitKey(0)
gauss = cv2.GaussianBlur(src_img,(5,5),6)
gray = cv2.cvtColor(gauss,cv2.COLOR_BGR2GRAY)
cv2.imshow('img',gray)
cv2.waitKey(0)

#Below code creates a track bar to set the parameters for canny edge detection. I have hard coded the value for users convinence 

# cv2.namedWindow('Edge Detect')
# cv2.createTrackbar('Thresh1','Edge Detect',0,255,nothing)
# cv2.createTrackbar('Thresh2','Edge Detect',0,255,nothing)
# t1 = t2 = 0
# while(1):
#     canny = cv2.Canny(gray,t1,t2)
#     cv2.imshow('Edge Detect',canny)
#     k = cv2.waitKey(1) & 0xFF
#     if k == 27:#Esc Key for terminating
#         break
#     t1 = cv2.getTrackbarPos('Thresh1','Edge Detect')
#     t2 = cv2.getTrackbarPos('Thresh2','Edge Detect')
t1=63
t2=95
canny = cv2.Canny(gray,t1,t2)
cv2.imshow('Edge Detect',canny)
cv2.waitKey(0) 

-1

In [13]:
cv2.imshow('canny',canny)
cv2.waitKey(0)
canny.shape

(370, 1929)

In [14]:
img2 = canny.copy()
#img2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
cv2.imshow('cropped',img2)
cv2.waitKey(0)
a1 = np.zeros((img2.shape[0],img2.shape[1]),np.uint8)
for c in range(img2.shape[1]):
    for r in range(img2.shape[0]):
        if img2[r,c]==255:
            a1[r,c]=255
            break
cv2.imshow('Upper Pixels', a1)
cv2.waitKey(0)
for c in range(img2.shape[1]-1,0,-1):
    for r in range(img2.shape[0]-1,0,-1):
        if img2[r,c]==255:
            a1[r,c]=255
            break
cv2.imshow('Lower pixels', a1)
cv2.waitKey(0)
cv2.imwrite('canny extracted.jpg',a1)
print(img2.shape)

(370, 1929)


In [15]:
img2 = img2[:,1000:1400]
img3 = img2.copy()
a = np.zeros((img3.shape[0],img3.shape[1]),np.uint8)
for c in range(img3.shape[1]):
    for r in range(img3.shape[0]):
        if img3[r,c]==255:
            a[r,c]=255
            break
cv2.imshow('Upper Pixels', a)
cv2.waitKey(0)
for c in range(img3.shape[1]-1,0,-1):
    for r in range(img3.shape[0]-1,0,-1):
        if img3[r,c]==255:
            a[r,c]=255
            break
cv2.imshow('Lower pixels', a)
cv2.waitKey(0)
cv2.imwrite('pixel extraction.jpg',a)

True

In [16]:
cv2.imshow('img',a)
cv2.waitKey(0)
for i in range(1,a.shape[0]-1):
    for j in range(1,a.shape[1]-1):
        if a[i][j]==255 and a[i-1][j]==0 and a[i+1][j]==0 and a[i][j+1]==0 and a[i][j-1]==0 and a[i-1][j-1]==0 and a[i+1][j-1]==0 and a[i+1][j+1]==0 and a[i-1][j+1]==0:
            a[i][j]=0

for i in range(1,a.shape[0]-1):
    for j in range(1,a.shape[1]-1):
        if a[i][j]==0 and a[i-1][j]==255 and a[i+1][j]==255 and a[i][j+1]==255 and a[i][j-1]==255 and a[i-1][j-1]==255 and a[i+1][j-1]==255 and a[i+1][j+1]==255 and a[i-1][j+1]==255:
            a[i][j]=255

cv2.imwrite('noiseless.jpg',a)

True

In [17]:
img4 = a.copy()
for r in range(img4.shape[0]):
    if 255 in img4[r,:]:
        min2 = r
        break
for r in range(img4.shape[0]-1,-1,-1):
    if 255 in img4[r,:]:
        max2 = r
        break
print(min2)
print(max2)
mid = int((min2+max2)/2)
print(mid)

107
249
178


In [18]:
img4[int(mid),:] = 255
cv2.imshow('midline',img4)
cv2.waitKey(0)

-1

In [19]:
upper = []
for col in range(img4.shape[1]):
    for row in range(mid):
        if int(img4[row,col]) == 255:
            tup = (row,col)
            upper.append(tup)
            tup = ()

#print(upper)  

In [20]:
lower = []
for col in range(img4.shape[1]-1,-1,-1):
    for row in range(mid+1,img4.shape[0]):
        if int(img4[row,col]) == 255:
            tup = (row,col)
            lower.append(tup)
            tup = ()

#print(lower)  

In [21]:
direction_upper = []
for x in range(len(upper)-1):
    if upper[x][0] > upper[x+1][0]:
        tup = (1,upper[x][0],upper[x][1])
        direction_upper.append(tup)
        tup = ()
    elif upper[x][0] < upper[x+1][0]:
        tup = (0,upper[x][0],upper[x][1])
        direction_upper.append(tup)
        tup = ()
    else:
        continue

#print(direction_upper)

In [22]:
crest_upper = []
root_upper = []
for x in range(len(direction_upper)-4):
    if direction_upper[x][0] == 1 and direction_upper[x+1][0] == 0 and direction_upper[x+2][0] == 0 and direction_upper[x+3][0] == 0 and direction_upper[x+4][0] == 0:
        tup = (direction_upper[x][1],direction_upper[x][2])
        crest_upper.append(tup)
        tup = ()
    elif direction_upper[x][0] == 0 and direction_upper[x+1][0] == 1 and direction_upper[x+2][0] == 1:
        tup = (direction_upper[x][1],direction_upper[x][2])
        root_upper.append(tup)
        tup = ()

In [23]:
print(crest_upper)
print(root_upper)

[(109, 21), (109, 50), (109, 81), (108, 112), (109, 167), (108, 200), (108, 232), (108, 260), (109, 287), (109, 315), (109, 347), (109, 378)]
[(122, 8), (122, 38), (122, 71), (122, 97), (121, 126), (122, 156), (123, 187), (122, 216), (122, 246), (122, 277), (122, 307), (121, 336), (121, 366)]


In [24]:
img5 = img4.copy()
for coord in crest_upper:
    cv2.circle(img5,(coord[1],coord[0]),5,155,1)
cv2.imshow('crestpoints',img5)
cv2.waitKey(0)

-1

In [25]:
img6 = img4.copy()
for coord in root_upper:
    cv2.circle(img6,(coord[1],coord[0]),5,155,1)
cv2.imshow('rootpoints',img6)
cv2.waitKey(0)

-1

In [26]:
(Y1,X1) = crest_upper[0]
(Y2,X2) = crest_upper[-1]

In [27]:
upper_crest_group = []
upper_root_group = []
for coord in upper:
    (Y3,X3) = coord
    pd = (((X2-X1)*(Y1-Y3)) - ((X1-X3)*(Y2-Y1)))/m.sqrt((X2-X1)**2 + (Y2-Y1)**2)
    if pd<1 and pd>-1:
        tup = (Y3,X3)
        upper_crest_group.append(tup)
        tup = ()

(Y1,X1) = root_upper[0]
(Y2,X2) = root_upper[-1]
for coord in upper:
    (Y3,X3) = coord
    pd = (((X2-X1)*(Y1-Y3)) - ((X1-X3)*(Y2-Y1)))/m.sqrt((X2-X1)**2 + (Y2-Y1)**2)
    if pd<1 and pd>-1:
        tup = (Y3,X3)
        upper_root_group.append(tup)
        tup = ()

In [28]:
img7 = img4.copy()
for coord in upper_crest_group:
    cv2.circle(img7,(coord[1],coord[0]),5,155,1)
cv2.imshow('crestpoints',img7)
cv2.waitKey(0)

img8 = img4.copy()
for coord in upper_root_group:
    cv2.circle(img8,(coord[1],coord[0]),5,155,1)
cv2.imshow('rootpoints',img8)
cv2.waitKey(0)

-1

In [29]:
print(len(upper_crest_group))
print(len(upper_root_group))

40
47


In [30]:
group = 1
upper_crest_clustered = []
upper_crest_clustered.append((upper_crest_group[0][0],upper_crest_group[0][1],group))
for i in range(len(upper_crest_group)-1):
    (Y1,X1) = upper_crest_group[i]
    (Y2,X2) = upper_crest_group[i+1]
    pd = m.sqrt((X2-X1)**2 + (Y2-Y1)**2)
    if pd<15:
        upper_crest_clustered.append((Y2,X2,group))
    else:
        group = group+1
        upper_crest_clustered.append((Y2,X2,group))

In [31]:
list2 = []
for i in range(len(upper_crest_clustered)):
    list2.append(upper_crest_clustered[i][2])
list2 = list(set(list2))
list2

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [32]:
list3 = []
final_upper_crests = []
for i in list2:
    for coord in upper_crest_clustered:
        if coord[2] == i:
            list3.append(coord)
    cd1 = list3[0]
    cd2 = list3[-1]
    mpo = int((cd1[1]+cd2[1])/2)
    for x in range(img4.shape[0]):
        if img4[x,mpo]==255:
            final_upper_crests.append((x,mpo))
            list3 = []
            break
print(final_upper_crests)    

[(109, 0), (108, 25), (108, 55), (108, 84), (107, 114), (109, 138), (108, 173), (107, 203), (108, 232), (107, 262), (108, 292), (108, 322), (108, 352), (108, 382)]


In [33]:
img9 = img4.copy()
for coord in final_upper_crests:
    cv2.circle(img9,(coord[1],coord[0]),4,200,1)
cv2.imshow('new_crests',img9)
cv2.waitKey(0)

-1

In [34]:
group1 = 1
upper_root_clustered = []
upper_root_clustered.append((upper_root_group[0][0],upper_root_group[0][1],group1))
for i in range(len(upper_root_group)-1):
    (Y1,X1) = upper_root_group[i]
    (Y2,X2) = upper_root_group[i+1]
    pd = m.sqrt((X2-X1)**2 + (Y2-Y1)**2)
    if pd<10:
        upper_root_clustered.append((Y2,X2,group1))
    else:
        group1 = group1+1
        upper_root_clustered.append((Y2,X2,group1))

In [35]:
list6 = []
for i in range(len(upper_root_clustered)):
    list6.append(upper_root_clustered[i][2])
list6 = list(set(list6))
list6

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [36]:
list7 = []
final_upper_roots = []
for i in list6:
    for coord in upper_root_clustered:
        if coord[2] == i:
            list7.append(coord)
    cd1 = list7[0]
    cd2 = list7[-1]
    mpo = int((cd1[1]+cd2[1])/2)
    for x in range(mid-1,0,-1):
        if img4[x,mpo]==255:
            final_upper_roots.append((x,mpo))
            list7 = []
            break
print(final_upper_roots)    

[(124, 11), (123, 40), (123, 70), (124, 99), (123, 129), (123, 158), (124, 188), (123, 217), (122, 246), (122, 277), (122, 307), (122, 337), (122, 367), (122, 397)]


In [37]:
img10 = img4.copy()
for coord in final_upper_crests:
    cv2.circle(img10,(coord[1],coord[0]),4,200,1)

for coord in final_upper_roots:
    cv2.circle(img10,(coord[1],coord[0]),4,200,1)

cv2.imshow('final_image',img10)
cv2.waitKey(0)

-1

In [38]:
img11 = img10.copy()
cv2.line(img11,(final_upper_crests[0][1],final_upper_crests[0][0]),(final_upper_crests[-1][1],final_upper_crests[-1][0]),200,1)
cv2.line(img11,(final_upper_roots[0][1],final_upper_roots[0][0]),(final_upper_roots[-1][1],final_upper_roots[-1][0]),200,1)
cv2.imshow('lines',img11)
cv2.waitKey(0)

-1

In [39]:
direction_lower = []
for x in range(len(lower)-1):
    if lower[x][0] > lower[x+1][0]:
        tup = (1,lower[x][0],lower[x][1])
        direction_lower.append(tup)
        tup = ()
    elif lower[x][0] < lower[x+1][0]:
        tup = (0,lower[x][0],lower[x][1])
        direction_lower.append(tup)
        tup = ()
    else:
        continue

#print(direction_lower)

In [40]:
root_lower = []
crest_lower = []
for x in range(len(direction_lower)-4):
    if direction_lower[x][0] == 1 and direction_lower[x+1][0] == 0 and direction_lower[x+2][0] == 0 and direction_lower[x+3][0] == 0 and direction_lower[x+4][0] == 0:
        tup = (direction_lower[x][1],direction_lower[x][2])
        root_lower.append(tup)
        tup = ()
    elif direction_lower[x][0] == 0 and direction_lower[x+1][0] == 1 and direction_lower[x+2][0] == 1:
        tup = (direction_lower[x][1],direction_lower[x][2])
        crest_lower.append(tup)
        tup = ()


In [41]:
print(root_lower)
print(crest_lower)

[(235, 386), (235, 355), (235, 324), (235, 296), (234, 264), (234, 236), (233, 205), (234, 176), (234, 147), (235, 117), (234, 87), (234, 57), (233, 27)]
[(246, 395), (247, 371), (247, 343), (247, 313), (247, 284), (246, 252), (247, 219), (246, 194), (246, 161), (247, 130), (247, 102), (247, 74), (247, 45), (248, 11)]


In [42]:
(Y1,X1) = root_lower[0]
(Y2,X2) = root_lower[-1]
lower_root_group = []
for coord in lower:
    (Y3,X3) = coord
    pd = (((X2-X1)*(Y1-Y3)) - ((X1-X3)*(Y2-Y1)))/m.sqrt((X2-X1)**2 + (Y2-Y1)**2)
    if pd<1 and pd>-1:
        tup = (Y3,X3)
        lower_root_group.append(tup)
        tup = ()

(Y1,X1) = crest_lower[0]
(Y2,X2) = crest_lower[-1]
lower_crest_group = []
for coord in lower:
    (Y3,X3) = coord
    pd = (((X2-X1)*(Y1-Y3)) - ((X1-X3)*(Y2-Y1)))/m.sqrt((X2-X1)**2 + (Y2-Y1)**2)
    if pd<1 and pd>-1:
        tup = (Y3,X3)
        lower_crest_group.append(tup)
        tup = ()

In [43]:
group2 = 1
lower_root_clustered = []
lower_root_clustered.append((lower_root_group[0][0],lower_root_group[0][1],group2))
for i in range(len(lower_root_group)-1):
    (Y1,X1) = lower_root_group[i]
    (Y2,X2) = lower_root_group[i+1]
    pd = m.sqrt((X2-X1)**2 + (Y2-Y1)**2)
    if pd<7:
        lower_root_clustered.append((Y2,X2,group2))
    else:
        group2 = group2+1
        lower_root_clustered.append((Y2,X2,group2))

In [44]:
list10 = []
for i in range(len(lower_root_clustered)):
    list10.append(lower_root_clustered[i][2])
list10 = list(set(list10))
list10

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [45]:
list11 = []
final_lower_roots = []
for i in list10:
    for coord in lower_root_clustered:
        if coord[2] == i:
            list11.append(coord)
    cd1 = list11[0]
    cd2 = list11[-1]
    mpo = int((cd1[1]+cd2[1])/2)
    for x in range(mid+1,img4.shape[0]):
        if img4[x,mpo]==255:
            final_lower_roots.append((x,mpo))
            list11 = []
            break
print(final_lower_roots)   

[(234, 384), (234, 353), (234, 322), (234, 293), (233, 263), (233, 233), (232, 204), (233, 173), (233, 144), (233, 114), (233, 85), (233, 55), (232, 26)]


In [46]:
img12 = img10.copy()
for coord in final_lower_roots:
    cv2.circle(img12,(coord[1],coord[0]),4,200,1)
cv2.imshow('final_lower_roots',img12)
cv2.waitKey(0)

-1

In [47]:
group3 = 1
lower_crest_clustered = []
lower_crest_clustered.append((lower_crest_group[0][0],lower_crest_group[0][1],group3))
for i in range(len(lower_crest_group)-1):
    (Y1,X1) = lower_crest_group[i]
    (Y2,X2) = lower_crest_group[i+1]
    pd = m.sqrt((X2-X1)**2 + (Y2-Y1)**2)
    if pd<11:
        lower_crest_clustered.append((Y2,X2,group3))
    else:
        group3 = group3+1
        lower_crest_clustered.append((Y2,X2,group3))

In [48]:
list14 = []
for i in range(len(lower_crest_clustered)):
    list14.append(lower_crest_clustered[i][2])
list14 = list(set(list14))
list14

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [49]:
list15 = []
final_lower_crests = []
for i in list14:
    for coord in lower_crest_clustered:
        if coord[2] == i:
            list15.append(coord)
    cd1 = list15[0]
    cd2 = list15[-1]
    mpo = int((cd1[1]+cd2[1])/2)
    for x in range(img4.shape[0]-1,-1,-1):
        if img4[x,mpo]==255:
            final_lower_crests.append((x,mpo))
            list15 = []
            break
#print(final_lower_crests)   

In [50]:
img4.shape

(370, 400)

In [51]:
img13 = img12.copy()
for coord in final_lower_crests:
    cv2.circle(img13,(coord[1],coord[0]),4,200,1)
cv2.imshow('final_lower_crests',img13)
cv2.waitKey(0)
cv2.imwrite('root&crests.jpg',img13)

True

In [52]:
img14 = img13.copy()
cv2.line(img14,(final_upper_crests[0][1],final_upper_crests[0][0]),(final_upper_crests[-1][1],final_upper_crests[-1][0]),200,1)
cv2.line(img14,(final_upper_roots[0][1],final_upper_roots[0][0]),(final_upper_roots[-1][1],final_upper_roots[-1][0]),200,1)
cv2.line(img14,(final_lower_roots[0][1],final_lower_roots[0][0]),(final_lower_roots[-1][1],final_lower_roots[-1][0]),200,1)
cv2.line(img14,(final_lower_crests[0][1],final_lower_crests[0][0]),(final_lower_crests[-1][1],final_lower_crests[-1][0]),200,1)
cv2.imshow('lines',img14)
cv2.waitKey(0)

-1

In [53]:
pitch_list = []
for i in range(len(final_upper_crests)-1):
    X1 = final_upper_crests[i][0]
    Y1 = final_upper_crests[i][1]
    X2 = final_upper_crests[i+1][0]
    Y2 = final_upper_crests[i+1][1]
    dist = m.sqrt((X2-X1)**2 + (Y2-Y1)**2)
    pitch_list.append(dist)
for i in range(len(final_upper_roots)-1):
    X1 = final_upper_roots[i][0]
    Y1 = final_upper_roots[i][1]
    X2 = final_upper_roots[i+1][0]
    Y2 = final_upper_roots[i+1][1]
    dist = m.sqrt((X2-X1)**2 + (Y2-Y1)**2)
    pitch_list.append(dist)
for i in range(len(final_lower_roots)-1):
    X1 = final_lower_roots[i][0]
    Y1 = final_lower_roots[i][1]
    X2 = final_lower_roots[i+1][0]
    Y2 = final_lower_roots[i+1][1]
    dist = m.sqrt((X2-X1)**2 + (Y2-Y1)**2)
    pitch_list.append(dist)
for i in range(len(final_lower_crests)-1):
    X1 = final_lower_crests[i][0]
    Y1 = final_lower_crests[i][1]
    X2 = final_lower_crests[i+1][0]
    Y2 = final_lower_crests[i+1][1]
    dist = m.sqrt((X2-X1)**2 + (Y2-Y1)**2)
    pitch_list.append(dist)

print("Mean pitch in pixel distance:"+str(sum(pitch_list)/len(pitch_list))+" pixels")    

Mean pitch in pixel distance:29.57759456701729 pixels


In [54]:
conversion = cv2.imread('mm_measure6.jpg',1)
cv2.imshow('crop',conversion)
cv2.waitKey(0)
conversion.shape

(872, 1193, 3)

In [55]:
#1193 pixels = 50mm
pixels_per_metric = 50/1193
mean_pitch = sum(pitch_list)/len(pitch_list)
print("Pitch of the screw:"+str(pixels_per_metric * mean_pitch)+" mm")

Pitch of the screw:1.2396309541918396 mm


In [56]:
temp = canny.copy()

for c in range(temp.shape[1]):
    if 255 in temp[:,c]:
        min2 = c
        break
for c in range(temp.shape[1]-1,-1,-1):
    if 255 in temp[:,c]:
        max2 = c
        break
print(min2)
print(max2)
print('Length of the screw:'+str((max2-min2)*pixels_per_metric)+' mm')

11
1907
Length of the screw:79.46353730092206 mm


In [57]:
depth_list = []

Y1,X1 = final_upper_roots[0][0] , final_upper_roots[0][1]
Y2,X2 = final_upper_roots[-1][0] , final_upper_roots[-1][1]
for coord in final_upper_crests:
    Y3,X3 = coord[0],coord[1]
    pd = (((X2-X1)*(Y1-Y3)) - ((X1-X3)*(Y2-Y1)))/m.sqrt((X2-X1)**2 + (Y2-Y1)**2)
    depth_list.append(abs(pd))

Y1,X1 = final_lower_crests[0][0] , final_lower_crests[0][1]
Y2,X2 = final_lower_crests[-1][0] , final_lower_crests[-1][1]
for coord in final_lower_roots:
    Y3,X3 = coord[0],coord[1]
    pd = (((X2-X1)*(Y1-Y3)) - ((X1-X3)*(Y2-Y1)))/m.sqrt((X2-X1)**2 + (Y2-Y1)**2)
    depth_list.append(abs(pd))
mean_depth = sum(depth_list)/len(depth_list)
print("Depth of screw threads:"+str(mean_depth*pixels_per_metric)+" mm")

Depth of screw threads:0.6187646990857084 mm


In [58]:
major_dia_list = []
Y1,X1 = final_lower_crests[0][0] , final_lower_crests[0][1]
Y2,X2 = final_lower_crests[-1][0] , final_lower_crests[-1][1]
for coord in final_upper_crests:
    Y3,X3 = coord[0],coord[1]
    pd = (((X2-X1)*(Y1-Y3)) - ((X1-X3)*(Y2-Y1)))/m.sqrt((X2-X1)**2 + (Y2-Y1)**2)
    major_dia_list.append(abs(pd))
mean_major_dia = sum(major_dia_list)/len(major_dia_list)
print("Major Diameter of screw:"+str(mean_major_dia*pixels_per_metric)+" mm")

Major Diameter of screw:5.851208853791424 mm


In [59]:
minor_dia_list = []
Y1,X1 = final_lower_roots[0][0] , final_lower_roots[0][1]
Y2,X2 = final_lower_roots[-1][0] , final_lower_roots[-1][1]
for coord in final_upper_roots:
    Y3,X3 = coord[0],coord[1]
    pd = (((X2-X1)*(Y1-Y3)) - ((X1-X3)*(Y2-Y1)))/m.sqrt((X2-X1)**2 + (Y2-Y1)**2)
    minor_dia_list.append(abs(pd))
mean_minor_dia = sum(minor_dia_list)/len(minor_dia_list)
print("Minor Diameter of screw:"+str(mean_minor_dia*pixels_per_metric)+" mm")

Minor Diameter of screw:4.618683649874743 mm


In [60]:
print((mean_major_dia*pixels_per_metric) - 2*(mean_depth*pixels_per_metric))

4.613679455620007


In [61]:
included_angle_list = []
for i in range(min(len(final_lower_roots),len(final_lower_crests))-1):
    (y1,x1) = (final_lower_crests[i][0],final_lower_crests[i][1])
    (y2,x2) = (final_lower_crests[i+1][0],final_lower_crests[i+1][1])
    (y3,x3) = (final_lower_roots[i+1][0],final_lower_roots[i+1][1])
    m1 = (y1-y3)/(x1-x3)
    m2 = (y2-y3)/(x2-x3)
    tan_theta = abs((m1-m2)/(1+m1*m2))
    included_angle_list.append(tan_theta)
included_angle_list = [m.atan(y)*180/m.pi for y in included_angle_list]
print(included_angle_list)
print("Thread angle of the thread:"+str(sum(included_angle_list)/len(included_angle_list))+u"\u00b0")

[25.432641426936428, 24.2584121015626, 25.74356761730138, 26.565051177077997, 24.590117166196016, 29.989364816050283, 22.75097634278764, 27.71850162818336, 23.535800945779528, 27.71850162818336, 26.565051177077997, 28.830968312390375]
Thread angle of the thread:26.141579528293914°


In [62]:
img2 = canny.copy()
img2 = img2[:,1500:]
cv2.imshow('canny',img2)
cv2.waitKey(0)

-1

In [63]:
img3 = img2.copy()
a = np.zeros((img3.shape[0],img3.shape[1]))
for c in range(img3.shape[1]):
    for r in range(img3.shape[0]):
        if img3[r,c]==255:
            a[r,c]=255
            break
cv2.imshow('Upper Pixels', a)
cv2.waitKey(0)
for c in range(img3.shape[1]-1,0,-1):
    for r in range(img3.shape[0]-1,0,-1):
        if img3[r,c]==255:
            a[r,c]=255
            break
cv2.imshow('Lower pixels', a)
cv2.waitKey(0)

-1

In [64]:
cv2.imshow('thinned',a)
cv2.waitKey(0)

-1

In [65]:
img4 = a.copy()
print(img4.shape)
for i in range(img4.shape[0]-1,-1,-1):
    if 255 in img4[i,:]:
        i1 = i
        break
print(i1)

(370, 429)
330


In [66]:
img4 = a.copy()
lines = []
for j in range(img4.shape[1]):
    if img4[i1,j] == 255:
        lines.append((i1,j))
for coord in lines:
    cv2.circle(img4,(coord[1],coord[0]),5,200,1)
cv2.imshow('image',img4)
cv2.waitKey(0)

-1

In [67]:
img5 = a.copy()
print(img5.shape)
for i in range(img4.shape[0]):
    if 255 in img4[i,:]:
        i2 = i
        break
print(i2)

(370, 429)
31


In [68]:
img5 = a.copy()
lines2 = []
for j in range(img5.shape[1]):
    if img5[i2,j] == 255:
        lines2.append((i2,j))
while len(lines2)<=3:
    lines2 = []
    i2 = i2+1
    for j in range(img5.shape[1]):
        if img5[i2,j] == 255:
            lines2.append((i2,j))
    
for coord in lines2:
    cv2.circle(img5,(coord[1],coord[0]),5,200,1)
cv2.imshow('image',img5)
cv2.waitKey(0)

-1

In [69]:
collar_dia_list = []
y1,x1 = lines[0][0],lines[0][1]
y2,x2 = lines[-1][0],lines[-1][1]
for coord in lines2:
    y3,x3 = coord[0],coord[1]
    pd = (((X2-X1)*(Y1-Y3)) - ((X1-X3)*(Y2-Y1)))/m.sqrt((X2-X1)**2 + (Y2-Y1)**2)
    collar_dia_list.append(abs(pd))
    
print("The collar diameter of the screw:"+str(sum(collar_dia_list)/len(collar_dia_list)*pixels_per_metric)+" mm")

The collar diameter of the screw:4.697019146909225 mm


In [70]:
img6 = a.copy()
cv2.imshow('image',img6)
cv2.waitKey(0)
print(img6.shape)
img6 = img6[:,250:]
cv2.imshow('image',img6)
cv2.waitKey(0)
print(img6.shape)

(370, 429)
(370, 179)


In [71]:
head = cv2.imread('screw_head7.jpg',1)
head = cv2.cvtColor(head,cv2.COLOR_BGR2GRAY)
cv2.imshow('head',head)
cv2.waitKey(0)
head.shape
_,thresh1 = cv2.threshold(head,150,250,cv2.THRESH_BINARY)
cv2.imshow('head',thresh1)
cv2.waitKey(0)

-1

In [72]:
head2 = head.copy()

#Below code creates a track bar to set the parameters for canny edge detection. I have hard coded the value for users convinence 

# cv2.namedWindow('Shape Detect')
# cv2.createTrackbar('Thresh1','Shape Detect',0,255,nothing)
# cv2.createTrackbar('Thresh2','Shape Detect',0,255,nothing)
# t3 = t4 = 0
# while(1):
#     #_,thresh1 = cv2.threshold(head,t3,t4,cv2.THRESH_TOZERO)
#     head_canny = cv2.Canny(head2,t3,t4)
#     cv2.imshow('Shape Detect',head_canny)
#     z = cv2.waitKey(1) & 0xFF
#     if z == 27:#Esc Key for terminating
#         break
#     t3 = cv2.getTrackbarPos('Thresh1','Shape Detect')
#     t4 = cv2.getTrackbarPos('Thresh2','Shape Detect')
# cv2.destroyAllWindows()

t3=128
t4=49
head_canny = cv2.Canny(head2,t3,t4)
cv2.imshow('Shape Detect',head_canny)
cv2.waitKey(0)

-1

In [73]:
fin_img = head_canny.copy()
for y in range(fin_img.shape[1]):
    for x in range(fin_img.shape[0]):
        if (fin_img[x,y])==255:
            min_x,min_y = x,y
            break
        
for y in range(fin_img.shape[1]-1,-1,-1):
    for x in range(fin_img.shape[0]):
        if (fin_img[x,y])==255:
            max_x,max_y = x,y
            break
    
print((max_x,max_y))
print((min_x,min_y))
cop = fin_img.copy()
cv2.line(cop,(min_y,min_x),(max_y,max_x),200,1)
cv2.imshow('lined',cop)
cv2.waitKey(0)
print(cop.shape)
cv2.imwrite('acrcor.jpg',cop)
ppm_meas = cv2.imread('mm_measure7.jpg',1)
print(ppm_meas.shape)
#456 pixels = 12mm
ppm = 6/ppm_meas.shape[1] 
print("Across corner of the screw:"+str(m.sqrt((max_x-min_x)**2 + (max_y-min_y)**2) * ppm)+" mm")
for i in range(cop.shape[0]-1,-1,-1):
    if 255 in cop[i,:]:
        i1 = i
        break
print(i1)
img15 = cop.copy()
print(img15.shape)
lines = []
for j in range(img15.shape[1]):
    if img15[i1,j] == 255:
        lines.append((i1,j))
while len(lines)<=3:
    lines = []
    i1 = i1+1
    for j in range(img15.shape[1]):
        if img15[i1,j] == 255:
            lines.append((i1,j))
for coord in lines:
    cv2.circle(img15,(coord[1],coord[0]),5,200,1)
cv2.imshow('image',img15)
cv2.waitKey(0)
cv2

for i in range(cop.shape[0]):
    if 255 in cop[i,:]:
        i2 = i
        break
#print(i2)
img16 = img15.copy()
lines2 = []
for j in range(img16.shape[1]):
    if img16[i2,j] == 255:
        lines2.append((i2,j))
while len(lines2)<=3:
    lines2 = []
    i2 = i2+1
    for j in range(img16.shape[1]):
        if img16[i2,j] == 255:
            lines2.append((i2,j))
    
for coord in lines2:
    cv2.circle(img16,(coord[1],coord[0]),5,200,1)
cv2.imshow('image',img16)
cv2.waitKey(0)
cv2.imwrite('acrflt.jpg',img16)

across_flat_list = []

Y1,X1 = lines[0][0] , lines[0][1]
Y2,X2 = lines[-1][0] , lines[-1][1]
for coord in lines2:
    Y3,X3 = coord[0],coord[1]
    pd = (((X2-X1)*(Y1-Y3)) - ((X1-X3)*(Y2-Y1)))/m.sqrt((X2-X1)**2 + (Y2-Y1)**2)
    across_flat_list.append(abs(pd))
    
mean_across_flat = sum(across_flat_list)/len(across_flat_list)
print("Across flat of screw:"+str(mean_across_flat*ppm)+" mm")

(251, 13)
(253, 538)
(521, 556)
(1202, 266, 3)
Across corner of the screw:11.842191191954619 mm
486
(521, 556)
Across flat of screw:10.330827067669173 mm


In [74]:
pitchby4 = (sum(pitch_list)/len(pitch_list))/4

print(final_upper_crests[0],final_upper_roots[0])

(109, 0) (124, 11)


In [75]:
cv2.imshow('image',img10)
cv2.waitKey(0)

-1

In [76]:
temp_img = img10.copy()
temp_img = cv2.cvtColor(temp_img,cv2.COLOR_GRAY2BGR)
cv2.circle(temp_img,(int((final_upper_crests[0][1]+final_upper_roots[0][1])/2),int((final_upper_crests[0][0]+final_upper_roots[0][0])/2)),3,(0,255,0),1)
cv2.imshow('temp_img',temp_img)
cv2.waitKey(0)
temp_img = cv2.cvtColor(temp_img,cv2.COLOR_BGR2GRAY)

In [77]:
p1 = []
for x in range(final_upper_crests[0][0]+1,int((final_upper_crests[0][0]+final_upper_roots[0][0])/2)):
    for y in range(final_upper_crests[0][1]+1,int((final_upper_crests[0][1]+final_upper_roots[0][1])/2)):
        if int(temp_img[x][y]) == 255:
            p1.append((x,y))
print(p1)

p1 = []
for x in range(int((final_upper_crests[0][0]+final_upper_roots[0][0])/2)+1,final_upper_roots[0][0]):
    for y in range(int((final_upper_crests[0][1]+final_upper_roots[0][1])/2)+1,final_upper_roots[0][1]):
        if int(temp_img[x][y]) == 255:
            p1.append((x,y))
print(p1)

[(110, 2)]
[(121, 7)]


In [78]:
cv2.imwrite('thinned.jpg',a)
cv2.imwrite('midp.jpg',temp_img)

True

In [79]:
# Outputs
print("Pitch of the screw:"+str(pixels_per_metric * mean_pitch)+" mm")
print('Length of the screw:'+str((max2-min2)*pixels_per_metric)+' mm')
print("Depth of screw threads:"+str(mean_depth*pixels_per_metric)+" mm")
print("Major Diameter of screw:"+str(mean_major_dia*pixels_per_metric)+" mm")
print("Minor Diameter of screw:"+str(mean_minor_dia*pixels_per_metric)+" mm")
print("Thread angle of the thread:"+str(sum(included_angle_list)/len(included_angle_list))+u"\u00b0")
print("The collar diameter of the screw:"+str(sum(collar_dia_list)/len(collar_dia_list)*pixels_per_metric)+" mm")
print("Across corner of the screw:"+str(m.sqrt((max_x-min_x)**2 + (max_y-min_y)**2) * ppm)+" mm")
print("Across flat of screw:"+str(mean_across_flat*ppm)+" mm")

Pitch of the screw:1.2396309541918396 mm
Length of the screw:79.46353730092206 mm
Depth of screw threads:0.6187646990857084 mm
Major Diameter of screw:5.851208853791424 mm
Minor Diameter of screw:4.618683649874743 mm
Thread angle of the thread:26.141579528293914°
The collar diameter of the screw:4.697019146909225 mm
Across corner of the screw:11.842191191954619 mm
Across flat of screw:10.330827067669173 mm
